## Setup Python Environment 

In [2]:
%%capture
# Setup environment
# !apt-get -qq install xxd
!sudo apt-get update -y
!sudo apt-get install -y xxd
!pip install pandas numpy matplotlib
%tensorflow_version 2.x
!pip install tensorflow

In [5]:
 ! python --version 
 print(f"TensorFlow version = {tf.__version__}\n")

Python 3.9.16
TensorFlow version = 2.12.0



In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [6]:
from google.colab import files
uploaded = files.upload()
correct_shots = pd.read_csv("test1.csv", names=['ax', 'ay', 'az', 'gx', 'gy', 'gz'])
uploaded = files.upload()
incorrect_shots = pd.read_csv("test2.csv", names=['ax', 'ay', 'az', 'gx', 'gy', 'gz'])
print("correct shape:", correct_shots.shape)
print("incorrect shape:", incorrect_shots.shape)

Saving test1.csv to test1.csv


Saving test2.csv to test2.csv
correct shape: (1727, 6)
incorrect shape: (946, 6)


In [7]:
# Lable 1 for correct shot
correct_shots["label"] = 1
print("correct shape:", correct_shots.shape)
# Lable zero for incorrect shot
incorrect_shots["label"] = 0
print("incorrect shape:", incorrect_shots.shape)


# combine two datasets
data = pd.concat([correct_shots, incorrect_shots], ignore_index=True)
print(data.shape)

# Pre Process Data
def extract_features_and_labels(data):
    features = data.drop("label", axis=1)  # Delete column "lable"
    labels = data["label"]                # mark "label" as tag/lable
    return features, labels

# Devide data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=42)

# Extract data and lable
train_X, train_y = extract_features_and_labels(train_data)
val_X, val_y = extract_features_and_labels(val_data)
test_X, test_y = extract_features_and_labels(test_data)

correct shape: (1727, 7)
incorrect shape: (946, 7)
(2673, 7)


In [8]:
print(f"train_X shape: {train_X.shape}")
print(f"train_y shape: {train_y.shape}")
print(f"val_X shape: {val_X.shape}")
print(f"val_y shape: {val_y.shape}")
print(f"test_X shape: {test_X.shape}")
print(f"test_y shape: {test_y.shape}")

train_X shape: (1603, 6)
train_y shape: (1603,)
val_X shape: (535, 6)
val_y shape: (535,)
test_X shape: (535, 6)
test_y shape: (535,)


In [9]:
train_X = train_X.values
train_y = train_y.values
val_X = val_X.values
val_y = val_y.values
test_X = test_X.values
test_y = test_y.values

In [12]:
model = Sequential([
    Dense(32, activation='relu', input_shape=(6,)),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_X, train_y, epochs=100, batch_size=32, validation_data=(val_X, val_y))
test_loss, test_accuracy = model.evaluate(test_X, test_y)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

Epoch 1/100
51/51 [==============================] - 2s 8ms/step - loss: 7.0686 - accuracy: 0.5596 - val_loss: 2.0554 - val_accuracy: 0.6579
Epoch 2/100
51/51 [==============================] - 0s 5ms/step - loss: 4.2411 - accuracy: 0.6407 - val_loss: 1.5136 - val_accuracy: 0.7196
Epoch 3/100
51/51 [==============================] - 0s 5ms/step - loss: 3.2404 - accuracy: 0.6606 - val_loss: 1.3219 - val_accuracy: 0.7271
Epoch 4/100
51/51 [==============================] - 0s 5ms/step - loss: 2.4859 - accuracy: 0.6588 - val_loss: 1.0466 - val_accuracy: 0.7196
Epoch 5/100
51/51 [==============================] - 0s 5ms/step - loss: 1.9686 - accuracy: 0.6457 - val_loss: 0.8467 - val_accuracy: 0.7047
Epoch 6/100
51/51 [==============================] - 0s 5ms/step - loss: 1.6373 - accuracy: 0.6588 - val_loss: 0.7018 - val_accuracy: 0.7196
Epoch 7/100
51/51 [==============================] - 0s 5ms/step - loss: 1.3061 - accuracy: 0.6937 - val_loss: 0.5887 - val_accuracy: 0.7252
Epoch 8/100
5

Test

In [13]:
test_loss, test_accuracy = model.evaluate(test_X, test_y)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

17/17 [==============================] - 0s 4ms/step - loss: 0.2799 - accuracy: 0.8822
Test loss: 0.2799239456653595, Test accuracy: 0.8822429776191711


In [14]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to disk
open("gesture_model.tflite", "wb").write(tflite_model)
  
import os
basic_model_size = os.path.getsize("gesture_model.tflite")
print("Model is %d bytes" % basic_model_size)

Model is 5168 bytes


In [15]:
!echo "const unsigned char model[] = {" > /content/model.h
!cat gesture_model.tflite | xxd -i      >> /content/model.h
!echo "};"                              >> /content/model.h

import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
# print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")

Header file, model.h, is 31,904 bytes.


In [16]:
files.download('model.h')
print('model.h Downloaded')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

model.h Downloaded
